# Instructions

This code is largely based on Chris's code. We all need to use a consistent NLP process (that way we only need to explain one process in our report). I decided to include stopwords for more natural conversational language processing. You will need to manually modify how to import the data into the notebook, file paths, etc.

This takes in raw text data and outputs processed data as a csv. This does not filter or upload to Chroma DB (next steps). For now, just save processed data on your own computer.

# Import Libraries

In [3]:
!pip install jsonlines

In [1]:
import numpy as np
import pandas as pd
import jsonlines
import os
import io
import time
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import ast
from langdetect import detect

from summarizer import Summarizer
import random

import re, string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

#nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/joeymeyer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/joeymeyer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/joeymeyer/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/joeymeyer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/joeymeyer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Google Drive

In [3]:
from google.colab import drive
drive.mount('drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# drive.flush_and_unmount()

In [ ]:
%cd /content/drive/'My Drive'/'SJSU Classes'/'*2023 Fall DATA298A'/'BERT data'
!ls

/content/drive/My Drive/SJSU Classes/*2023 Fall DATA298A/BERT data
bertopic_test.pkl   train_0.jsonl  train_2.jsonl  train_4.jsonl
processed_data.csv  train_1.jsonl  train_3.jsonl  train_5.jsonl


In [ ]:
# Define the folder containing the JSONL files
cwd = os.getcwd()

## Sonia's Machine

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

2023-11-15 18:04:53.772596: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 18:04:53.772625: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 18:04:53.772653: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 18:04:53.778772: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


2023-11-15 18:04:56.879308: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-15 18:04:56.879495: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-15 18:04:56.913048: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

# File Paths

In [3]:
%cd /media/joeymeyer/970-evo-plus/Sonia/bertproj

/media/joeymeyer/970-evo-plus/Sonia/bertproj


/media/joeymeyer/970-evo-plus/Sonia/bertproj/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
%ls

bertproj/  data_viz/  reddit/  venv/


In [7]:
cwd = os.getcwd()
cwd

'/media/joeymeyer/970-evo-plus/Sonia/bertproj'

## Get Data - UltraChat

In [13]:
%cd /media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj/ultrachat
%ls

/media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj/ultrachat
train_0.jsonl*  train_3.jsonl*  train_6.jsonl*  train_9.jsonl*
train_1.jsonl*  train_4.jsonl*  train_7.jsonl*  ultrachat_1_output/
train_2.jsonl*  train_5.jsonl*  train_8.jsonl*  ultrachat_rest_output/


In [14]:
cwd = os.getcwd()
cwd

'/media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj/ultrachat'

In [15]:
source = 'ultrachat_rest'
output_dir = os.path.join(os.getcwd(), f'{source}_output')
output_dir

'/media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj/ultrachat/ultrachat_rest_output'

In [31]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [6]:
# Initialize a count variable
jsonl_count = 0

# Iterate through the files in the directory
for filename in os.listdir(cwd):
    if filename.endswith(".jsonl"):
        jsonl_count += 1

In [17]:
# I used huggingface ultrachat data which is jsonlines file.
# Manually update this to get data however you have it and in whatever format.

with tqdm(total=jsonl_count, unit="row", desc="Processing", dynamic_ncols=True) as pbar: #pbar.update(1) ###
    
    # Initialize an empty list to store data from each JSONL file
    data = []
    
    # Iterate through each JSONL file in the folder
    for filename in os.listdir(cwd):
        if filename.endswith(".jsonl"):
            file_path = os.path.join(cwd, filename)
    
            # Open the JSONL file and read its contents
            with jsonlines.open(file_path, 'r') as reader:
                try:
                    for item in reader:
                        # Add the filename as a new key-value pair in each JSON object
                        item['filename'] = filename ### MAKE SURE TO INCLUDE FILENAME
                        data.append(item)
                        pbar.update(1)
                except jsonlines.InvalidLineError as e:
                    print(f"Skipped invalid line in {file_path}: {e}")
    
            # break #remove eventually, just taking one file for testing
    
    # Convert the list of JSON objects into a Pandas DataFrame
    df = pd.DataFrame(data)
    # Now you can work with the DataFrame 'df' containing data from all JSONL files

# Update task with timing information
task = f'Reading JSON lines for {df.shape[0]} conversations'
print(f"Elapsed time for {task}: {pbar.format_interval(pbar.last_print_t - pbar.start_t)}")

Processing: 150000row [00:03, 42469.10row/s]

Elapsed time for Reading JSON lines for 150000 conversations: 00:03


In [18]:
df.head()

,id,data,filename
0,0,[How can cross training benefit groups like ru...,train_0.jsonl
1,1,[Are there any particular physical benefits to...,train_0.jsonl
2,2,[What percentage of the Earth's surface is cov...,train_0.jsonl
3,3,[How does language translation technology impa...,train_0.jsonl
4,4,[What is the most popular smartphone brand the...,train_0.jsonl


In [12]:
df.drop('id', axis=1, inplace=True)
df['source'] = source
df = df[df.columns[::-1]]

# df = df.head() # here I'm just taking 5 for quick processing, you will want to do your whole dataset

In [13]:
df

,source,filename,data
0,ultrachat,train_0.jsonl,[How can cross training benefit groups like ru...
1,ultrachat,train_0.jsonl,[Are there any particular physical benefits to...
2,ultrachat,train_0.jsonl,[What percentage of the Earth's surface is cov...
3,ultrachat,train_0.jsonl,[How does language translation technology impa...
4,ultrachat,train_0.jsonl,[What is the most popular smartphone brand the...
...,...,...,...
1319820,ultrachat,train_9.jsonl,[Could you please give me more information abo...
1319821,ultrachat,train_9.jsonl,[Administer the Research Enhancement and Facul...
1319822,ultrachat,train_9.jsonl,[What is the purpose of the public viewing eve...
1319823,ultrachat,train_9.jsonl,[Factory and headquarters of Foulds Macaroni C...


In [14]:
df.size

3959475

## Get Data - Dolly

In [21]:
%cd /media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj/dolly

/media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj/dolly


/media/joeymeyer/970-evo-plus/Sonia/bertproj/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [22]:
%ls

databricks-dolly-15k_travel.pkl*  dolly_output/


In [23]:
source = 'dolly'
output_dir = os.path.join(os.getcwd(),f'{source}_output')

In [35]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [24]:
import pickle

# Later, you can load the results
with open('databricks-dolly-15k_travel.pkl', 'rb') as file:
    df = pickle.load(file)

# Access the loaded results
df.head()

,instruction,context,response,category
0,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin A...",Virgin Australia commenced services on 31 Augu...,closed_qa
1,"Alice's parents have three daughters: Amy, Jes...",,The name of the third daughter is Alice,open_qa
2,"Given a reference text about Lollapalooza, whe...",Lollapalooza /ˌlɒləpəˈluːzə/ (Lolla) is an ann...,Lollapalooze is an annual musical festival hel...,closed_qa
3,Who is Thomas Jefferson?,"Thomas Jefferson (April 13, 1743 – July 4, 182...","Thomas Jefferson (April 13, 1743 – July 4, 182...",information_extraction
4,Who was Kyle Van Zyl playing against when he s...,Van Zyl joined the Eastern Province Kings Acad...,Kyle Van Zyl was playing against Boland U21 wh...,closed_qa


In [19]:
df['combined'] = df['instruction'].str.cat([df['context'], df['response']], sep=' ')

In [23]:
df['source'] = source
df = df[df.columns[::-1]]

# df = df.head() # here I'm just taking 5 for quick processing, you will want to do your whole dataset

In [24]:
df

,source,combined,category,response,context,instruction
0,dolly,When did Virgin Australia start operating? Vir...,closed_qa,Virgin Australia commenced services on 31 Augu...,"Virgin Australia, the trading name of Virgin A...",When did Virgin Australia start operating?
1,dolly,"Alice's parents have three daughters: Amy, Jes...",open_qa,The name of the third daughter is Alice,,"Alice's parents have three daughters: Amy, Jes..."
2,dolly,"Given a reference text about Lollapalooza, whe...",closed_qa,Lollapalooze is an annual musical festival hel...,Lollapalooza /ˌlɒləpəˈluːzə/ (Lolla) is an ann...,"Given a reference text about Lollapalooza, whe..."
3,dolly,Who is Thomas Jefferson? Thomas Jefferson (Apr...,information_extraction,"Thomas Jefferson (April 13, 1743 – July 4, 182...","Thomas Jefferson (April 13, 1743 – July 4, 182...",Who is Thomas Jefferson?
4,dolly,Who was Kyle Van Zyl playing against when he s...,closed_qa,Kyle Van Zyl was playing against Boland U21 wh...,Van Zyl joined the Eastern Province Kings Acad...,Who was Kyle Van Zyl playing against when he s...
...,...,...,...,...,...,...
5620,dolly,"What is geomorphometry? Geomorphometry, or ge...",open_qa,"Geomorphometry, or geomorphometrics, is the sc...",,What is geomorphometry?
5621,dolly,Tell me about the various road types in USA? ...,classification,The public road system in United States of Ame...,,Tell me about the various road types in USA?
5622,dolly,What is a laser and who created it? A laser is...,summarization,A laser is a device that emits light from an e...,A laser is a device that emits light through a...,What is a laser and who created it?
5623,dolly,How does GIS help in the real estate investmen...,general_qa,"Real estate investors depend on precise, accur...",,How does GIS help in the real estate investmen...


In [25]:
df.size

33750

## Get Data - Reddit

In [25]:
%cd /media/joeymeyer/970-evo-plus/Sonia/bertproj/reddit/reddit_output/most_recent

/media/joeymeyer/970-evo-plus/Sonia/bertproj/reddit/reddit_output/most_recent


/media/joeymeyer/970-evo-plus/Sonia/bertproj/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [26]:
%ls

all_reddit_2023-11-12_to_2023-11-13.csv*  final_topic_results.pkl*
final_bertopic_model.pkl*                 final_topic_visualization.png*
final_heatmap_visualization.png*          processed_data.csv*
final_hierarchy_visualization.png*        reddit_travel_df.csv*


In [27]:
source = 'reddit'
output_dir = os.getcwd()

In [30]:
df = pd.read_csv('all_reddit_2023-11-12_to_2023-11-13.csv')

pd.set_option('display.max_colwidth', 10)
# Access the loaded results
df.head()

,Unnamed: 0.1,Unnamed: 0,source,filename,title,selftext,creation_date,id,url,upvote_ratio,ups,downs,score,link_flair_css_class,comments
0,0,0,reddit,travel...,Passpo...,NOTE: ...,2023-0...,100t75r,https:...,0.99,536.0,0.0,536.0,question,['SPRI...
1,1,1,reddit,travel...,U.S. D...,U.S. D...,2023-1...,17bouw5,https:...,0.94,743.0,0.0,743.0,advice,['Yes....
2,2,2,reddit,travel...,[Updat...,First ...,2023-1...,17tvwy3,https:...,0.96,1296.0,0.0,1296.0,advice,['Whil...
3,3,3,reddit,travel...,Just m...,I’m Am...,2023-1...,17twyhu,https:...,0.88,881.0,0.0,881.0,question,['Of c...
4,4,4,reddit,travel...,We nee...,I’ve b...,2023-1...,17tm34k,https:...,0.91,383.0,0.0,383.0,advice,['I ge...


In [48]:
df.shape

(9708, 15)

In [49]:
# Function to detect the language of a text
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

df['language'] = df['title'].apply(detect_language)
# Filter out rows with non-English text
df = df[df['language'] == 'en']
# Drop the 'language' column (optional)
df = df.drop('language', axis=1)

In [50]:
df.shape

(7210, 15)

In [51]:
df = df[df['upvote_ratio'] >= 0.8]

In [52]:
df.shape

(5276, 15)

In [ ]:
df.columns

In [53]:
# Convert the strings back to lists using ast.literal_eval
df['comments'] = df['comments'].apply(ast.literal_eval)
df['comments'] = df['comments'].apply(lambda x: x[:20] if isinstance(x, list) else x)
df = df[df['comments'].apply(len) > 0]

In [62]:
df.shape

(4313, 15)

In [70]:
def list_to_string(lst):
    return '[' + ', '.join(map(str, lst)) + ']'

# Apply the function to the 'comments' column
df['comments'] = df['comments'].apply(lambda x: list_to_string(x))

In [72]:
df['comments'].isnull().sum()

0

In [64]:
df = df.dropna(subset=['comments'])

In [65]:
df['selftext'] = df['selftext'].replace('[nan]', np.nan)
df['selftext'] = df['selftext'].replace('[]', np.nan)
df['selftext'].isnull().sum()

2226

In [102]:
# df = df.dropna(subset=['selftext'])

In [76]:
# Define a custom function to combine 'title', 'selftext', and 'comments' while skipping NaN values
def combine_non_nan(row):
    non_nan_values = [val for val in row if not pd.isna(val)]
    return ' '.join(map(str, non_nan_values))

# Apply the custom function to each row
df['combined'] = df[['title', 'selftext', 'comments']].apply(combine_non_nan, axis=1)

In [74]:
df['combined'] = df['title'].str.cat([df['selftext'], df['comments']], sep=' ')

In [78]:
df['combined'] = df['combined'].replace('[nan]', np.nan)
df['combined'] = df['combined'].replace('[]', np.nan)
df['combined'].isnull().sum()

0

In [79]:
df = df.drop(columns=['Unnamed: 0.1','Unnamed: 0'])
df = df.reset_index(drop=True)
df

,source,filename,title,selftext,creation_date,id,url,upvote_ratio,ups,downs,score,link_flair_css_class,comments,combined
0,reddit,travel_hot_100_2023-11-12,Passport Questions & Issues Megathread (2023),NOTE: October 2023 **If the US Government has ...,2023-01-01 12:56:19,100t75r,https://www.reddit.com/r/travel/comments/100t7...,0.99,536.0,0.0,536.0,question,[SPRING BREAK RUSH HAS STARTED. AS OF TODAY PR...,Passport Questions & Issues Megathread (2023) ...
1,reddit,travel_hot_100_2023-11-12,"U.S. Department of State - ""Worldwide Caution""",U.S. Department of State issued a new travel a...,2023-10-19 10:41:36,17bouw5,https://www.reddit.com/r/travel/comments/17bou...,0.94,743.0,0.0,743.0,advice,[Yes. They are routinely issued when there is ...,"U.S. Department of State - ""Worldwide Caution""..."
2,reddit,travel_hot_100_2023-11-12,[Update] Jewelry stolen from luggage,"First of all, I want to thank everyone who too...",2023-11-12 14:40:44,17tvwy3,https://www.reddit.com/r/travel/comments/17tvw...,0.96,1296.0,0.0,1296.0,advice,[While I’m glad you’re receiving full compensa...,[Update] Jewelry stolen from luggage First of ...
3,reddit,travel_hot_100_2023-11-12,Just me or is the US now far and away the most...,I’m American and everything from hotel prices/...,2023-11-12 15:27:10,17twyhu,https://www.reddit.com/r/travel/comments/17twy...,0.88,881.0,0.0,881.0,question,[Of course. I live in San Diego and it blows m...,Just me or is the US now far and away the most...
4,reddit,travel_hot_100_2023-11-12,We need to be more supportive of each other IR...,I’ve been traveling solo for a little bit and ...,2023-11-12 07:05:56,17tm34k,https://www.reddit.com/r/travel/comments/17tm3...,0.91,383.0,0.0,383.0,advice,"[I get tired of the ""I'm a traveller not a tou...",We need to be more supportive of each other IR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4308,reddit,adventures_top_1000_2023-11-12,Best city and itinerary in Southeast Asia for ...,I (18M from USA) am looking to head to Southea...,2023-02-01 20:21:32,10rgjc9,https://www.reddit.com/r/adventures/comments/1...,1.00,1.0,0.0,1.0,NaN,"[SE Asia is super easy to get around, so I rec...",Best city and itinerary in Southeast Asia for ...
4309,reddit,adventures_top_1000_2023-11-12,Riverboarding Adventure,NaN,2023-01-26 13:58:54,10m3skn,https://youtu.be/qVti_oKTURY,1.00,1.0,0.0,1.0,NaN,[Riverboarding in the summer of 2022 on the Lo...,Riverboarding Adventure [Riverboarding in the ...
4310,reddit,adventures_top_1000_2023-11-12,Walkabout ideas?,Recent life events have left me needing a soci...,2023-01-22 07:34:05,10imakx,https://www.reddit.com/r/adventures/comments/1...,1.00,1.0,0.0,1.0,NaN,[I'm 42 and enjoy solo backcountry hiking as w...,Walkabout ideas? Recent life events have left ...
4311,reddit,adventures_top_1000_2023-11-12,Riverside Camping on Algoma's Whiskey Lake Loop,NaN,2023-01-22 06:26:40,10iku8b,https://youtu.be/wkDCh09Zcfc,1.00,1.0,0.0,1.0,NaN,"[Spent the night fishing, cooking and having s...",Riverside Camping on Algoma's Whiskey Lake Loo...


In [80]:
df.shape

(4313, 14)

# Define Functions

In [81]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    text=text.strip()
    text=re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    #text = re.sub(r'\[[0-9]*\]',' ',text)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    #text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text

# # STOPWORD REMOVAL
# def stopword(string):
#     a= [i for i in string.split() if i not in STOPWORDS]
#     return ' '.join(a)

# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(preprocess(string)).replace('\n','')
    # return lemmatizer(stopword(preprocess(string)))

# Data Pre-processing

In [110]:
# Function to join the sentences in a list into a single string
def combine_sentences(sentence_list):
    return ' '.join(sentence_list)

# Apply the function to each row in the DataFrame
df['combined'] = df['data'].apply(lambda x: combine_sentences(x)) #manually update column name

In [27]:
# #batch processing saved 1 min
# #may be useful if other datasets are too large and hit RAM issues
# #otherwise leave it

# # Define batch size
# batch_size = 30000

# # Calculate the number of batches
# num_batches = (len(df) + batch_size - 1) // batch_size
# batch_results = []

# # Create a single progress bar outside the loop
# with tqdm_notebook(total=len(df), unit="row", desc="Processing", dynamic_ncols=True) as pbar:
#     # Process data in batches
#     for i in range(num_batches): 
#         start_idx = i * batch_size
#         end_idx = min((i + 1) * batch_size, len(df))
#         batch_df = df.iloc[start_idx:end_idx]

#         results = batch_df['combined'].apply(finalpreprocess)        
#         batch_results.append(results)
#         # Access and use the results as needed for each batch
#         pbar.update(len(batch_df))  # Update the progress bar based on the batch size

# elapsed_time = pbar.format_interval(pbar.last_print_t - pbar.start_t)

# task = f'NLP preprocessing for {df.shape[0]} rows of text'
# print(f"Elapsed time for {task}: {elapsed_time}")

Processing:   0%|                                                        | 0/150000 [00:00<?, ?row/s]

Elapsed time for NLP preprocessing for 150000 rows of text: 1:07:32


In [41]:
# results_series = pd.concat(batch_results, axis=0)
# df['cause_clean'] = results_series
# df.head()

,source,filename,data,combined,cause_clean
0,ultrachat,train_0.jsonl,[How can cross training benefit groups like ru...,How can cross training benefit groups like run...,how can cross training benefit group like runn...
1,ultrachat,train_0.jsonl,[Are there any particular physical benefits to...,Are there any particular physical benefits to ...,be there any particular physical benefit to mi...
2,ultrachat,train_0.jsonl,[What percentage of the Earth's surface is cov...,What percentage of the Earth's surface is cove...,what percentage of the earth s surface be cove...
3,ultrachat,train_0.jsonl,[How does language translation technology impa...,How does language translation technology impac...,how do language translation technology impact ...
4,ultrachat,train_0.jsonl,[What is the most popular smartphone brand the...,What is the most popular smartphone brand thes...,what be the most popular smartphone brand thes...


In [82]:
start_time = time.time()

tqdm.pandas(desc="Processing")
df['cause_clean'] = df['combined'].progress_apply(finalpreprocess)

end_time = time.time()
elapsed_time = end_time - start_time
#update task
task = f'NLP preprocessing for {df.shape[0]} rows of text'
if elapsed_time < 60:
    print(f"Elapsed time for {task}: {elapsed_time:.1f} seconds")
elif elapsed_time < 60 * 10:
    print(f"Elapsed time for {task}: {elapsed_time/60:.2f} minutes")
else:
    print(f"Elapsed time for {task}: {elapsed_time/60/60:.2f} hours") 

Processing: 100%|███████████████████████████| 4313/4313 [01:24<00:00, 51.31it/s]

Elapsed time for NLP preprocessing for 4313 rows of text: 1.40 minutes


In [36]:
# df = pd.read_csv('processed_data.csv')
# df = df.head(1000)

In [83]:
def summarize_sample(text, model):
    words = text.split()
    max_size = len(words)
    sample_size = 100
    if sample_size > max_size:
        return text
    else:
        random_sample = " ".join(random.sample(words, sample_size))
        summary = model(random_sample, ratio=0.2)
        if summary != "":
            return summary
        else:
            random_sample = " ".join(random.sample(words, 75))
            summary = model(random_sample, ratio=0.2)
            return summary

In [84]:
start_time = time.time()

model = Summarizer()

tqdm.pandas(desc="Processing")
df['bert_summary'] = df['cause_clean'].progress_apply(lambda x: summarize_sample(x, model))

end_time = time.time()
elapsed_time = end_time - start_time
#update task
task = f'NLP preprocessing for {df.shape[0]} rows of text'
if elapsed_time < 60:
    print(f"Elapsed time for {task}: {elapsed_time:.1f} seconds")
elif elapsed_time < 60 * 60:
    print(f"Elapsed time for {task}: {elapsed_time/60:.2f} minutes")
else:
    print(f"Elapsed time for {task}: {elapsed_time/60/60:.2f} hours") 

Processing: 100%|███████████████████████████| 4313/4313 [01:31<00:00, 46.95it/s]

Elapsed time for NLP preprocessing for 4313 rows of text: 1.59 minutes


In [85]:
df['bert_summary'].replace('', np.nan, inplace=True)
round(df['bert_summary'].isnull().sum()/df.shape[0],2)

0.0

In [86]:
### CHECK HERE TO SEE THAT NULL ISN'T TOO HIGH

In [87]:
df.shape

(4313, 16)

In [88]:
df = df.dropna(subset=['bert_summary'])
df.shape

(4312, 16)

In [89]:
df = df.reset_index(drop=True)
df

,source,filename,title,selftext,creation_date,id,url,upvote_ratio,ups,downs,score,link_flair_css_class,comments,combined,cause_clean,bert_summary
0,reddit,travel_hot_100_2023-11-12,Passport Questions & Issues Megathread (2023),NOTE: October 2023 **If the US Government has ...,2023-01-01 12:56:19,100t75r,https://www.reddit.com/r/travel/comments/100t7...,0.99,536.0,0.0,536.0,question,[SPRING BREAK RUSH HAS STARTED. AS OF TODAY PR...,Passport Questions & Issues Megathread (2023) ...,passport question issue megathread 2023 note o...,the look that sol or use 9 give good u of stat...
1,reddit,travel_hot_100_2023-11-12,"U.S. Department of State - ""Worldwide Caution""",U.S. Department of State issued a new travel a...,2023-10-19 10:41:36,17bouw5,https://www.reddit.com/r/travel/comments/17bou...,0.94,743.0,0.0,743.0,advice,[Yes. They are routinely issued when there is ...,"U.S. Department of State - ""Worldwide Caution""...",u s department of state worldwide caution u s ...,sept and a to potentially increase im s that i...
2,reddit,travel_hot_100_2023-11-12,[Update] Jewelry stolen from luggage,"First of all, I want to thank everyone who too...",2023-11-12 14:40:44,17tvwy3,https://www.reddit.com/r/travel/comments/17tvw...,0.96,1296.0,0.0,1296.0,advice,[While I’m glad you’re receiving full compensa...,[Update] Jewelry stolen from luggage First of ...,update jewelry steal from luggage first of all...,yourself our i airline i in and but sorry and ...
3,reddit,travel_hot_100_2023-11-12,Just me or is the US now far and away the most...,I’m American and everything from hotel prices/...,2023-11-12 15:27:10,17twyhu,https://www.reddit.com/r/travel/comments/17twy...,0.88,881.0,0.0,881.0,question,[Of course. I live in San Diego and it blows m...,Just me or is the US now far and away the most...,just me or be the u now far and away the most ...,also tax as dollar eat find less of esp everyw...
4,reddit,travel_hot_100_2023-11-12,We need to be more supportive of each other IR...,I’ve been traveling solo for a little bit and ...,2023-11-12 07:05:56,17tm34k,https://www.reddit.com/r/travel/comments/17tm3...,0.91,383.0,0.0,383.0,advice,"[I get tired of the ""I'm a traveller not a tou...",We need to be more supportive of each other IR...,we need to be more supportive of each other ir...,to no u while super me a the to and worldview ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4308,reddit,adventures_top_1000_2023-11-12,Best city and itinerary in Southeast Asia for ...,I (18M from USA) am looking to head to Southea...,2023-02-01 20:21:32,10rgjc9,https://www.reddit.com/r/adventures/comments/1...,1.00,1.0,0.0,1.0,NaN,"[SE Asia is super easy to get around, so I rec...",Best city and itinerary in Southeast Asia for ...,best city and itinerary in southeast asia for ...,with thailand can understandably with north ca...
4309,reddit,adventures_top_1000_2023-11-12,Riverboarding Adventure,NaN,2023-01-26 13:58:54,10m3skn,https://youtu.be/qVti_oKTURY,1.00,1.0,0.0,1.0,NaN,[Riverboarding in the summer of 2022 on the Lo...,Riverboarding Adventure [Riverboarding in the ...,riverboarding adventure riverboarding in the s...,riverboarding adventure riverboarding in the s...
4310,reddit,adventures_top_1000_2023-11-12,Walkabout ideas?,Recent life events have left me needing a soci...,2023-01-22 07:34:05,10imakx,https://www.reddit.com/r/adventures/comments/1...,1.00,1.0,0.0,1.0,NaN,[I'm 42 and enjoy solo backcountry hiking as w...,Walkabout ideas? Recent life events have left ...,walkabout idea recent life event have leave me...,resupplies and this look m different and take ...
4311,reddit,adventures_top_1000_2023-11-12,Riverside Camping on Algoma's Whiskey Lake Loop,NaN,2023-01-22 06:26:40,10iku8b,https://youtu.be/wkDCh09Zcfc,1.00,1.0,0.0,1.0,NaN,"[Spent the night fishing, cooking and having s...",Riverside Camping on Algoma's Whiskey Lake Loo...,riverside camp on algoma s whiskey lake loop s...,riverside camp on algoma s whiskey lake loop s...


# Save

In [90]:
df.to_csv(f"{output_dir}/processed_data.csv", index=False, escapechar='\\')

In [44]:
# df.to_csv(f"{output_dir}/processed_data.csv", index=False)
# update appropriate title here